# Random Schedule

Author: Steven M. Hernandez

The goal of this notebook is used to understand how random scheduling affects eavesdropper.

In [1]:
# 
# 0. Select experiment
# 
import ipywidgets as widgets
from datasets.experiment_group import ExperimentGroup

google_drive_directory = f"all_experiments/AntiEavesdrop/ttyUSB0"

experiment_group = ExperimentGroup(google_drive_directory, google_drive_directory.split("/")[-1])
experiment = experiment_group.experiments[0]

In [2]:
all_actions = [
    'doorway',
    'sitting desk',
    'standing desk',
    'closet',
    'empty room',
]

In [3]:
from incense import ExperimentLoader
from helpers.config import config

url = config("omniboard", "anti_eavesdrop.mongodbURI")
db_name = config("omniboard", "anti_eavesdrop.path").replace("/", "")

loader = ExperimentLoader(
    mongo_uri=url,
    db_name=db_name
)

def load_config(sacred_id):
    client = pymongo.MongoClient(url)
    sacred_db = client[db_name]
    runsDF = pd.DataFrame(list(sacred_db['runs'].find({
        'status': 'COMPLETED',
        '_id': sacred_id,
    }, {'captured_out': False})))
    
    assert(len(runsDF) == 1)
    
    return runsDF.iloc[0].config

In [4]:
from anti_eavesdrop.train_and_evaluate__multi_tx__random_schedule import TrainAndEvaluateMultiTXRandomSchedule
from helpers.config import config as c
from sacred.observers import MongoObserver
import ipywidgets as widgets
ta = widgets.Textarea(layout={'width': '100%'})
n = 0
optuna_labels = {}

tx_mac_addresses = {
    'A': '7C:9E:BD:4C:9C:5C',
    'B': 'C8:2B:96:8E:92:5C',
    'C': '94:B9:7E:C3:82:98',
    'D': '24:0A:C4:ED:30:50',
    'E': '94:B9:7E:C3:95:AC',
}

tx_best_config = {  # Update these sacred-ids accordingly
    'A': 2137,
    'B': 2243,
    'C': 1909,
    'D': 2113,
    'E': 2219,
}

for evaluate_txs in [
    'ABCDE',
]:
    for pca in [64]:
        for tx, mac in list(tx_mac_addresses.items()):
            sacred_id = tx_best_config[tx]
            config = load_config(sacred_id)
            config['mac_params'] = {
                'train_mac': mac,
                'evaluate_mac_list': [_mac for _tx, _mac in tx_mac_addresses.items() if _tx in evaluate_txs]
            }
            config['dataset_params']['interpolation'] = {
                'size_in_milliseconds': 50,
                'method': 'nearest',
            }

            label = f'anti_eavesdrop:random_schedule:train=TX-{tx}:evaluate=5TXs:{sacred_id}:pca={pca}'

            config['experiment_params']['experiment_group_label'] = label
            optuna_labels[label] = {
                'tx_train': tx,
                'tx_evaluate': evaluate_txs,
                'pca': pca,
            }
            s = TrainAndEvaluateMultiTXRandomSchedule().tsp(config, omniboard_config='anti_eavesdrop')
            ta.value += s
            n += 1

print(n)
display(ta)

5


Textarea(value='tsp docker-compose exec -T jupyter ipython -c "%run ../projects/anti_eavesdrop/train_and_evalu…

## Get Results

In [5]:
from incense import ExperimentLoader

url = c("omniboard", "anti_eavesdrop.mongodbURI")
db_name = c("omniboard", "anti_eavesdrop.path").replace("/", "")

loader = ExperimentLoader(
    mongo_uri=url,
    db_name=db_name
)

experiments = loader.find({
    'status': 'COMPLETED',
    'config.experiment_params.experiment_group_label': {'$in': list(optuna_labels.keys())},
})

output = []
for exp in experiments:
    label = exp.config['experiment_params']['experiment_group_label']
    d = {
#         'accuracy': exp.metrics['accuracy'].iloc[-1] * 100,
#         'label': label,
        'validation accuracy': exp.metrics['validation accuracy'].iloc[-1] * 100,
#             'tx': f"TX-{tx}",
        'tx_trained_on': optuna_labels[label]['tx_train'],
        'tx_evaluated_on': optuna_labels[label]['tx_evaluate'],
        'number_of_txs': len(optuna_labels[label]['tx_evaluate']),
        '_id': exp._id,
    }
    output.append(d)

df = pd.DataFrame(output)
df = df.sort_values(['tx_trained_on', 'tx_evaluated_on'])
print(len(df))
display(df)

df.to_csv('/tmp/experiments_server/random_schedule__TX-A.csv')
dl_btn('/tmp/experiments_server/random_schedule__TX-A.csv', 'random_schedule__TX-A.csv')

5


,validation accuracy,tx_trained_on,tx_evaluated_on,number_of_txs,_id
1,48.460412,A,ABCDE,5,2994
0,30.135301,B,ABCDE,5,2993
3,40.845442,C,ABCDE,5,2996
2,25.037259,D,ABCDE,5,2995
4,16.521738,E,ABCDE,5,2997


HTML(value='<html>\n    <head>\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    …

In [6]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=df.tx_trained_on,
    y=df['validation accuracy'],
    text=[f"{a:.2f}%" for a in df['validation accuracy']],
))
fig.update_layout(title="5-class Classifier")
fig.update_xaxes(title="Number of TX during Evaluation")
fig.update_yaxes(title="Accuracy", range=[0, 100])
fig.show('notebook_connected')

df[['tx_trained_on', 'validation accuracy']].to_csv('/tmp/experiments_server/eavesdropper.5_txs.csv')
dl_btn('/tmp/experiments_server/eavesdropper.5_txs.csv', 'eavesdropper.5_txs.csv')

HTML(value='<html>\n    <head>\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    …

In [9]:
from incense import ExperimentLoader
from helpers.config import config

url = config("omniboard", "anti_eavesdrop.mongodbURI")
db_name = config("omniboard", "anti_eavesdrop.path").replace("/", "")

loader = ExperimentLoader(
    mongo_uri=url,
    db_name=db_name
)

for i, row in df.iterrows():
    experiments = loader.find({
        # 'status': 'COMPLETED',
        '_id': row['_id'],
    })
    
    assert(len(experiments) == 1)
    
    print(experiments[0])
    class_map = experiments[0].artifacts['class_map.csv'].render()
    class_map = class_map.to_dict()['class_label']
    predictions = experiments[0].artifacts['predictions.test.99.csv'].render()
    
    N = len(class_map)
    Z = np.zeros([N, N])
    
    for i in range(N):
        for j in range(N):
            Z[i,j] = len(predictions[
                (predictions.y_true == i) &
                (predictions.y_pred == j)
            ])
            
    Z = 100 * (Z.T / Z.sum(axis=1)).T
            
    fig = px.imshow(
        Z,
        x=list(class_map.values()),
        y=list(class_map.values()),
        color_continuous_scale='inferno',
        zmax=100.0,
        zmin=0.0,
        aspect="auto",
    )
    fig.update_traces(text=Z, texttemplate="%{text:.2f}%")
    fig.update_xaxes(title="Predicted Class")
    fig.update_yaxes(title="True Class")
    fig.show('notebook_connected')
    
    pd.DataFrame(Z, columns=class_map.values(), index=class_map.values()).to_csv(f'/tmp/experiments_server/eavesdropper_rx_accuracy__cm__TX_{row["tx_trained_on"]}.csv')
    dl_btn(
        f'/tmp/experiments_server/eavesdropper_rx_accuracy__cm__TX_{row["tx_trained_on"]}.csv',
        f'eavesdropper_rx_accuracy__cm__TX_{row["tx_trained_on"]}.csv'
    )
    
    

Experiment(id=2994, name=train_and_evaluate__multi_tx__random_schedule)


HTML(value='<html>\n    <head>\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    …

Experiment(id=2993, name=train_and_evaluate__multi_tx__random_schedule)


HTML(value='<html>\n    <head>\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    …

Experiment(id=2996, name=train_and_evaluate__multi_tx__random_schedule)


HTML(value='<html>\n    <head>\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    …

Experiment(id=2995, name=train_and_evaluate__multi_tx__random_schedule)


HTML(value='<html>\n    <head>\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    …

Experiment(id=2997, name=train_and_evaluate__multi_tx__random_schedule)


HTML(value='<html>\n    <head>\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    …

In [10]:
from incense import ExperimentLoader
from helpers.config import config

url = config("omniboard", "anti_eavesdrop.mongodbURI")
db_name = config("omniboard", "anti_eavesdrop.path").replace("/", "")

loader = ExperimentLoader(
    mongo_uri=url,
    db_name=db_name
)

for i, row in df.iterrows():
    experiments = loader.find({
        # 'status': 'COMPLETED',
        '_id': row['_id'],
    })
    
    assert(len(experiments) == 1)
    
    print(experiments[0])
    print(experiments[0].artifacts)
    
    

Experiment(id=2994, name=train_and_evaluate__multi_tx__random_schedule)
{'confusion_matrix.0.png': ImageArtifact(name=confusion_matrix.0.png), 'confusion_matrix_validation.0.png': ImageArtifact(name=confusion_matrix_validation.0.png), 'class_map.csv': CSVArtifact(name=class_map.csv), 'predictions.train.0.csv': CSVArtifact(name=predictions.train.0.csv), 'predictions.test.0.csv': CSVArtifact(name=predictions.test.0.csv), 'confusion_matrix.99.png': ImageArtifact(name=confusion_matrix.99.png), 'confusion_matrix_validation.99.png': ImageArtifact(name=confusion_matrix_validation.99.png), 'predictions.train.99.csv': CSVArtifact(name=predictions.train.99.csv), 'predictions.test.99.csv': CSVArtifact(name=predictions.test.99.csv)}
Experiment(id=2993, name=train_and_evaluate__multi_tx__random_schedule)
{'confusion_matrix.0.png': ImageArtifact(name=confusion_matrix.0.png), 'confusion_matrix_validation.0.png': ImageArtifact(name=confusion_matrix_validation.0.png), 'class_map.csv': CSVArtifact(name=

In [11]:
from incense import ExperimentLoader
from helpers.config import config

url = config("omniboard", "anti_eavesdrop.mongodbURI")
db_name = config("omniboard", "anti_eavesdrop.path").replace("/", "")

loader = ExperimentLoader(
    mongo_uri=url,
    db_name=db_name
)

for i, row in df.iterrows():
    print(row['_id'])
    print(row['label'])
    experiments = loader.find({
        # 'status': 'COMPLETED',
        '_id': row['_id'],
    })

    print(experiments)
#         display(experiments[0].artifacts['confusion_matrix.99.png'].render())
    display(experiments[0].artifacts['confusion_matrix_validation.99.png'].render())
    for _ in range(9):
        print("=============")